# Working with Time Zones

## Handling local times

"Local time" is a very common concept that is irritatingly difficult to capture correctly. A naïve `datetime.datetime` in Python has a somewhat overloaded meaning in that it mostly represents an abstract datetime for use with calendrical calculations, but when used as a concrete time, it is interpreted as being in the system's local time zone.

For example:

In [1]:
from datetime import datetime, timezone
from dateutil import tz

from helper_functions import print_dt_tzinfo

In [2]:
from helper_functions import TZEnvContext  # Sets the local time zone during the context
                                           # This does not work on Windows

dt = datetime(2020, 1, 1, 12)
with TZEnvContext("America/Los_Angeles"):
    dt_la = dt.astimezone(timezone.utc)
    
with TZEnvContext("Asia/Tokyo"):
    dt_tok = dt.astimezone(timezone.utc)
    
print(dt_la)
print(dt_tok)

2020-01-01 20:00:00+00:00
2020-01-01 03:00:00+00:00


This works in the other direction as well:

In [3]:
dt = datetime(2020, 1, 1, 21, tzinfo=tz.gettz("Asia/Tokyo"))

with TZEnvContext("UTC"):
    print(dt.astimezone(None))

2020-01-01 12:00:00+00:00


These are not true local times, however, as they do not expose any information about the time zone through `tzname()` or `utcoffset()`:

In [4]:
print(datetime(2020, 1, 1, 12).utcoffset())

None


And you cannot perform comparisons or arithmetic between naive datetimes and aware datetimes:

In [5]:
NYC = tz.gettz("America/New_York")
try:
    datetime(2020, 1, 1, 12) - datetime(2020, 1, 1, 12, tzinfo=NYC)
except TypeError as e:
    print(repr(e))

TypeError("can't subtract offset-naive and offset-aware datetimes")


### `dateutil.tz.tzlocal`

The Python standard library provides hooks into the operating system's time zone information in the `time module`:

In [6]:
import time

with TZEnvContext("America/New_York"):
    print(f"tzname: {time.tzname}")
    print(f"timezone: {time.timezone}")
    print(f"altzone: {time.altzone}")

tzname: ('EST', 'EDT')
timezone: 18000
altzone: 14400


But there is no concrete local time object in the standard library, *so* `dateutil` has implemented one with `dateutil.tz.tzlocal`!

In [7]:
with TZEnvContext("America/Los_Angeles"):
    print("Los Angeles")
    print_dt_tzinfo(datetime(2020, 1, 1, 12, tzinfo=tz.tzlocal()))
print("")
with TZEnvContext("America/Chicago"):
    print("Chicago")
    print_dt_tzinfo(datetime(2020, 1, 1, 12, tzinfo=tz.tzlocal()))


Los Angeles
2020-01-01 12:00:00-0800
    tzname:   PST;      UTC Offset:  -8.00h;        DST:      0.0h

Chicago
2020-01-01 12:00:00-0600
    tzname:   CST;      UTC Offset:  -6.00h;        DST:      0.0h


This allows you to get a proper timezone-aware datetime in your system's locale.

### Changing local zone during a program's run

The system time zone changing during a program's run is *not* particularly well-supported operation, and you should avoid it if at all possible when working with local times: prefer to get the desired time zone from your users rather than from the system.

On Linux, it is necessary to call `time.tzset()` after any change to the the system time locale in order to see those changes reflected in the system. On Windows, `time.tzset()` does nothing, and restarting the interpreter is the only way to update the time zone from the perspective of the `time` function.

If your program is or could run on Windows, it is preferable to use the `dateutil.tz.tzwinlocal()` function to represent local times, as it queries the system registry directly.

On Windows, you can see this problem: 

```python
>>> dt = datetime(2014, 2, 11, 17, 0)

>>> print(dt.replace(tzinfo=tz.tzlocal()).tzname())
Eastern Standard Time

>>> print(dt.replace(tzinfo=tz.win.tzwinlocal()).tzname())
Eastern Standard Time

>>> with TZWinContext('Pacific Standard Time'):
...     print(dt.replace(tzinfo=tz.tzlocal()).tzname())
...     print(dt.replace(tzinfo=tz.win.tzwinlocal()).tzname())
```
```
Eastern Standard Time
Pacific Standard Time
```

However, in both cases (unlike the situation with all other time zone types), it is preferable to make a *new `tzinfo` object for every `datetime`*. The reason for this is that a certain amount of the behavior of `tzlocal` and `tzwinlocal` are set at construction time, so:

In [8]:
with TZEnvContext("America/New_York"):
    LOCAL = tz.tzlocal()

with TZEnvContext("America/Los_Angeles"):
    print(datetime(2014, 2, 11, 17, tzinfo=LOCAL))

with TZEnvContext("America/Chicago"):
    print(datetime(2014, 2, 11, 17, tzinfo=LOCAL))


2014-02-11 17:00:00-05:00
2014-02-11 17:00:00-05:00


Note that both of these are using the offset specified in the *original* time zone. What's worse is that because `tzlocal` does not have direct access to the function mapping `datetime` to offsets, if the system time zone changes, the offsets are baked in at runtime, but the dates of DST changes use the *current system offsets*, thus creating a hybrid object that is *really* wrong:

In [9]:
with TZEnvContext("America/New_York"):
    print(datetime(2019, 3, 20, 10, tzinfo=LOCAL))
    
with TZEnvContext("Europe/London"):
    print(datetime(2019, 3, 20, 10, tzinfo=LOCAL))
    print(datetime(2019, 3, 20, 10, tzinfo=tz.gettz("Europe/London")))

2019-03-20 10:00:00-04:00
2019-03-20 10:00:00-05:00
2019-03-20 10:00:00+00:00


If possible, I recommend avoiding using "local" time at all, and instead either convert to UTC eagerly (if dealing in absolute times) or have a user-configurable time zone mapping to either the IANA database or the Windows time zone settings.

And I will note that these particular kinds of bugs are probably incredibly rare with some mitigations, since it requires a non-Windows user to change their time zone during the lifetime of a `tzlocal` object to another time zone that has a different schedule for DST changes. It can happen, but it will probably not make your application useless if you don't handle it correctly.